In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import ignite
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import string
import math
from torch.autograd import Variable
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
torch.manual_seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
from sklearn.metrics import accuracy_score
from sklearn import linear_model, metrics, model_selection, ensemble
from sklearn.neural_network import MLPClassifier


In [50]:
def getDriveResult(driveId, gameId, df):
    first = df.query('driveIndex == @driveId & gameId == @gameId').iloc[[0]]
    final = df.query('driveIndex == @driveId & gameId == @gameId').iloc[[-1]]
    if(first['homeScore'].values[0]<final['homeScore'].values[0] or first['awayScore'].values[0]<final['awayScore'].values[0] ):
        return 1
    else:
        return 0

In [3]:
basic = pd.read_csv('./data/PBP - 2016 - Week 1.csv')
offense_stats = pd.read_csv('./data/Offense Stats 2016.csv')

In [53]:
basic.query('gameId == 400868876 & driveIndex == 1').iloc[[0]]

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,result
6,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass Reception,1,10,41,6,47,Seth Russell pass complete to KD Cannon for 6 ...,1.0


In [51]:
for index,row in basic.iterrows():
    basic.at[index,'result'] = getDriveResult(row['driveIndex'], row['gameId'], basic)

In [52]:
basic['result'].unique()

array([0., 1.])

In [55]:
basic = basic.dropna()

In [57]:
features = basic[['down', 'distance', 'yardLine']]
labels = basic[['result']]

In [59]:
X = features
y = labels
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)                    

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [63]:
pd.DataFrame(clf.predict_proba(features)).describe()

,0,1
count,16113.000000,16113.000000
mean,0.592331,0.407669
std,0.002793,0.002793
min,0.241686,0.407620
25%,0.592380,0.407620
50%,0.592380,0.407620
75%,0.592380,0.407620
max,0.592380,0.758314


In [64]:
[coef.shape for coef in clf.coefs_]

[(3, 5), (5, 2), (2, 1)]

In [65]:
df = basic.join(offense_stats.set_index('School'), on='homeTeam')

In [66]:
df = df.dropna()

In [68]:
features = df[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Rush Attempts', 'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
labels = df['result']

In [71]:
X = features
y = labels
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)                    

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [73]:
pd.DataFrame(clf.predict(features)).describe()

,0
count,11750.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [76]:
df.query('result == 1.0')

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,result,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
6,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass Reception,1,10,41,6,47,Seth Russell pass complete to KD Cannon for 6 ...,1.0,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
7,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,2,4,47,3,50,Shock Linwood run for 3 yds to the 50 yard line,1.0,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
8,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,2,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Penalty,3,1,50,-10,40,"Linwood,Shock rush for 3 yards to the NWST47 (...",1.0,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
9,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,3,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,3,11,40,42,82,Shock Linwood run for 42 yds to the NWSt 18 fo...,1.0,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
10,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,4,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass Reception,1,10,82,5,87,Seth Russell pass complete to Quan Jones for 5...,1.0,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16125,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,23,1,2653,Troy,TROY,2046,Austin Peay,PEAY,50,17,4,7:57,2016-09-04T07:41:15Z,Rush,2,3,20,17,37,Jabir Frye run for 17 yds to the Troy 37 for a...,1.0,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16126,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,23,2,2653,Troy,TROY,2046,Austin Peay,PEAY,50,17,4,7:57,2016-09-04T07:41:15Z,Rush,1,10,37,8,45,Kaleb Barker run for 8 yds to the Troy 45,1.0,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16127,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,23,3,2653,Troy,TROY,2046,Austin Peay,PEAY,50,17,4,7:57,2016-09-04T07:41:15Z,Pass Reception,2,2,45,1,46,Kaleb Barker pass complete to Jabir Frye for 1...,1.0,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16128,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,23,4,2653,Troy,TROY,2046,Austin Peay,PEAY,50,17,4,7:57,2016-09-04T07:41:15Z,Rush,3,1,46,4,50,B.J. Smith run for 4 yds to the 50 yard line f...,1.0,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
